In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
def evaluate_fairness(y_true, y_pred, sensitive_features):
        """
        Evaluates fairness of the final majority vote classifier over T_inner hypotheses
        on the test set.
        #NOTE: defined in the meta_algo file, but we chose:
        a0 := African-American (COMPAS), Female (Adult)
        a1 := Caucasian (COMPAS), Male (Adult)

        :return: list. subgroups in sensitive_features.
        :return: dict. recidivism_pct for each group.
        """
        groups = np.unique(sensitive_features.values)
        pos_count = {}
        dp_pct = {}
        eo_y0_pct = {}
        eo_y1_pct = {}
        
        for index, group in enumerate(groups):
            # Demographic Parity
            indices = {}
            indices[group] = sensitive_features.index[sensitive_features == group]
            dp_pct[group] = sum(y_pred[indices[group]])/len(indices[group])

            # Equalized Odds
            y1_indices = {}
            y0_indices = {}
            y1_indices[group] = sensitive_features.index[(sensitive_features == group) & (y_true == 1)]
            y0_indices[group] = sensitive_features.index[(sensitive_features == group) & (y_true == 0)]
            eo_y0_pct[group] = sum(y_pred[y0_indices[group]])/len(y0_indices[group])   
            eo_y1_pct[group] = sum(y_pred[y1_indices[group]])/len(y1_indices[group])
        
        gaps = {}
        group_metrics = {} # a dictionary of dictionaries

        gaps['dp'] = abs(dp_pct[groups[0]] - dp_pct[groups[1]])
        gaps['eo_y0'] = abs(eo_y0_pct[groups[0]] - eo_y0_pct[groups[1]])
        gaps['eo_y1'] = abs(eo_y1_pct[groups[0]] - eo_y1_pct[groups[1]])
        group_metrics['dp'] = dp_pct
        group_metrics['eo_y0'] = eo_y0_pct
        group_metrics['eo_y1'] = eo_y1_pct
        
        return groups, group_metrics, gaps

## Compas Dataset

In [ ]:
compas_train = pd.read_csv('./data/processed/compas/compas_train1_X.csv')
compas_test = pd.read_csv('./data/processed/compas/compas_test1_X.csv')

In [ ]:
compas_train.head(5)

In [ ]:
y_train = compas_train.pop('two_year_recid') 
y_test = compas_test.pop('two_year_recid')
sensitive_features_train = compas_train['race']
sensitive_features_test = compas_test['race']
X_train = compas_train
X_test = compas_test

In [ ]:
sensitive_features_test.head(5)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

## Adult Dataset

In [ ]:
adult_X = pd.read_csv('./data/adult_X.csv')
adult_y = pd.read_csv('./data/adult_y.csv')

adult_X.head(5)

X_train, X_test, y_train, y_test = train_test_split(adult_X, adult_y, test_size=0.2, random_state=42)
y_test = y_test.reset_index(drop=True)
y_test = y_test['income']
sensitive_features_train = X_train['sex']
sensitive_features_test = X_test['sex']

sensitive_features_train[sensitive_features_train < 0] = 0
sensitive_features_train[sensitive_features_train > 0] = 1
sensitive_features_train = sensitive_features_train.reset_index(drop=True)

sensitive_features_test[sensitive_features_test < 0] = 0
sensitive_features_test[sensitive_features_test > 0] = 1
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))
groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

## Law School Dataset

In [ ]:
lawschool_X = pd.read_csv('./data/lawschool_X.csv')
lawschool_y = pd.read_csv('./data/lawschool_y.csv')

X_train, X_test, y_train, y_test = train_test_split(lawschool_X, lawschool_y, test_size=0.2, random_state=42)
y_test = y_test.reset_index(drop=True)
y_test = y_test['bar1']
sensitive_features_train = X_train['race7']
sensitive_features_test = X_test['race7']

sensitive_features_train[sensitive_features_train < 0] = 0
sensitive_features_train[sensitive_features_train > 0] = 1
sensitive_features_train = sensitive_features_train.reset_index(drop=True)

sensitive_features_test[sensitive_features_test < 0] = 0
sensitive_features_test[sensitive_features_test > 0] = 1
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))
groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

## Communities Dataset

In [ ]:
communities_X = pd.read_csv('./data/communities_X.csv')
communities_y = pd.read_csv('./data/communities_y.csv')

X_train, X_test, y_train, y_test = train_test_split(communities_X, communities_y, test_size=0.2, random_state=42)
y_test = y_test.reset_index(drop=True)
y_test = y_test['ViolentCrimesPerPop']
sensitive_features_train = X_train['majority_white']
sensitive_features_test = X_test['majority_white']

sensitive_features_train[sensitive_features_train < 0] = 0
sensitive_features_train[sensitive_features_train > 0] = 1
sensitive_features_train = sensitive_features_train.reset_index(drop=True)

sensitive_features_test[sensitive_features_test < 0] = 0
sensitive_features_test[sensitive_features_test > 0] = 1
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))
groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

In [ ]:
communities_X.head[:10]